In [1]:
from utility import MonteCarlo
import pandas as pd

In [2]:
stocks = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA']

sim = MonteCarlo(stocks)

/Users/isaacyoho/Desktop/Coding/Data Science/MonteCarlo2/utility/monte_carlo.py:22: FutureWarning: YF.download() has changed argument auto_adjust default to True
  self.stock_data = yf.download(tickers, start=start_str, end=end_str)
[*********************100%***********************]  5 of 5 completed


In [3]:
sim.run_sim()



**************************************************
INITIAL CONDITIONS
**************************************************
Initial Investment: 10000
Years Simulated: 0.999315537303217
Weight of AAPL: 0.4164706234507731
Weight of MSFT: 0.33880431310026804
Weight of GOOGL: 0.09275935661058493
Weight of AMZN: 0.06716262370748828
Weight of TSLA: 0.08480308313088554
**************************************************


**************************************************
FINAL CONDITIONS
**************************************************
Mean Final Investment: 12941.77342942755
SD Final Investment: 4686.506571091915
Mean Annual Rate: 0.29440594963042943
Mean Total Return: 0.294177342942755
**************************************************


Displaying portfolio simulations as a matrix. Each row is a day, each column is a simulation.


[[ 9816.60447561 10494.81342155  9977.22390166 ... 10020.10089928
   9820.0018067  10124.55129511]
 [ 9856.97278427 10204.29807884 10135.15644407 ... 10236.9667

array([[ 9816.60447561, 10494.81342155,  9977.22390166, ...,
        10020.10089928,  9820.0018067 , 10124.55129511],
       [ 9856.97278427, 10204.29807884, 10135.15644407, ...,
        10236.96670404,  9848.36614231,  9838.40586873],
       [ 9724.47292479, 10018.75265508,  9877.08989433, ...,
        10355.36470423,  9681.89036076,  9873.6931308 ],
       ...,
       [13061.23359215, 12612.37146214,  9615.93736848, ...,
        14341.95823822, 12354.39820021,  8308.33961814],
       [12959.41894881, 12763.20415887,  9539.0336443 , ...,
        14238.31910364, 12354.25388264,  8129.00984743],
       [13190.02151905, 12715.85547376,  9723.70065565, ...,
        14265.86723899, 12351.14295672,  8053.23881663]], shape=(365, 100))